In [1]:
from glob import glob
import pandas as pd
# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')

# Create an empty list to store all files
away_team_info = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    away_team_info.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet')
    )

print(f"Total files loaded: {len(away_team_info)}") 
df_away_team_info = [pd.read_parquet(file) for file in away_team_info]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)
df_away_team_info.head()

Total files loaded: 59256


C:\Users\lavan_computer\AppData\Local\Temp\ipykernel_14020\103218961.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)


,match_id,name,slug,gender,user_count,residence,birthplace,height,weight,plays,...,period_2,period_3,period_4,period_5,period_1_tie_break,period_2_tie_break,period_3_tie_break,period_4_tie_break,period_5_tie_break,normal_time
0,11998445,Auger-Aliassime F.,auger-aliassime-felix,M,23318.0,"Monte Carlo, Monaco","Montreal, Canada",1.93,87.0,right-handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11998446,Cobolli F.,flavio-cobolli,M,5995.0,Rome,"Florence, Italy",1.83,71.0,right-handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11998447,Martínez P.,martinez-pedro,M,2621.0,"Valencia, Spain","Alzira, Spain",1.85,76.0,right-handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11998448,Muller A.,muller-alexandre,M,2443.0,France,"Poissy, France",1.83,73.0,right-handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11998449,Mayot H.,mayot-harold,M,1492.0,"Marly, France","Metz, France",1.78,78.0,right-handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
# Create an empty list to store all files
home_team_info = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    home_team_info.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet')
    )

print(f"Total files loaded: {len(home_team_info)}") 
df_home_team_info = [pd.read_parquet(file) for file in home_team_info]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)
df_home_team_info.head()

NameError: name 'date_folders' is not defined

In [ ]:
tour_info = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    tour_info.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/tournament*.parquet')
    )

print(f"Total files loaded: {len(tour_info)}") 
df_tour_info = [pd.read_parquet(file) for file in tour_info]
df_tour_info = pd.concat(df_tour_info, ignore_index=True)
df_tour_info.head()

In [ ]:
# Create an empty list to store all files
files_to_load_event = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_to_load_event.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/event*.parquet')
    )

print(f"Total files loaded: {len(files_to_load_event)}") 
event_info = [pd.read_parquet(file) for file in files_to_load_event]
event_info = pd.concat(event_info, ignore_index=True)
event_info.sample(5)